In [ ]:
melanoma = df[df.target == 1]
non_melanoma = df[df.target == 0]
y_mel = melanoma.target
X_mel = melanoma.drop(columns=['target'], axis=1)
y_non_mel = non_melanoma.target
X_non_mel = non_melanoma.drop(columns=['target'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_tr_m, X_tt_m, y_tr_m, y_tt_m = train_test_split(X_mel, y_mel, random_state=42, test_size=0.2)
X_tr_nm, X_tt_nm, y_tr_nm, y_tt_nm = train_test_split(X_non_mel, y_non_mel, random_state=42, test_size=0.2)

In [ ]:
mel_train = pd.concat([X_tr_m, y_tr_m], axis=1)
non_mel_train = pd.concat([X_tr_nm, y_tr_nm], axis=1)
mel_test = pd.concat([X_tt_m, y_tt_m], axis=1)
non_mel_test = pd.concat([X_tt_nm, y_tt_nm], axis=1)

train = pd.concat([mel_train, non_mel_train], axis=0)
test = pd.concat([mel_test, non_mel_test], axis=0)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
train['file'] = train.image_name.apply(lambda x: str(x) + ".jpg")
test['file'] = test.image_name.apply(lambda x: str(x) + ".jpg")
train.file

In [ ]:
train_ids = train.file
test_ids = test.file
train.head()
train.to_csv("data/train.csv")
test.to_csv("data/test.csv")

In [ ]:
jpeg_dir = 'images/jpeg'
dicom_dir = 'images/dicom'
jpeg_list = [file for file in os.listdir(jpeg_dir) if file.endswith('.jpg')]
dicom_list = [file for file in os.listdir(dicom_dir) if file.endswith('.dcm')]
train_ids = train.file
test_ids = test.file
train_ids

In [ ]:
import csv
import os

with open('data/train.csv', newline='') as csvfile:
    linereader = csv.reader(csvfile, delimiter=',')
    for row in linereader:
        name = row[0]
        try:
            os.('images/jpeg/' + name, 'images/jpeg/train/' + name)
            print(name + " moved to new folder.")
        except FileNotFoundError:
            pass 
        except FileExistsError:
            pass

In [ ]:
import csv
import os
import shutil

def main():
    csv_file = "data/train.csv"
    from_folder = "images/jpeg/"
    to_folder = "images/jpeg/train/"

    with open(csv_file, 'r', newline='') as f:
        reader = csv.reader(f)

        for row in reader:
            file_name = row[0]
            from_filename = os.path.join(from_folder, file_name)
            to_filename = os.path.join(to_folder, file_name)

            try:
                shutil.move(from_filename, to_filename)
                print("Moved - '{}' -> '{}'".format(from_filename, to_filename))
            except shutil.Error as e:
                print("Failed - '{}' -> '{}'".format(from_filename, to_filename))

main()

In [ ]:
import splitfolders
splitfolders.ratio(jpeg_dir, output="output", seed=42, ratio=(.8,0.1,0.1))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Converting DICOM

In [ ]:
import pydicom as dicom
import matplotlib.pyplot as plt
import os
import cv2
import PIL # optional
import pandas as pd
import csv

# make it True if you want in PNG format
PNG = True
# Specify the .dcm folder path
folder_path = "images/test"
# Specify the .jpg/.png folder path
jpg_folder_path = "images/png_test"
images_path = os.listdir(folder_path)
for n, image in enumerate(images_path):
    ds = dicom.dcmread(os.path.join(folder_path, image))
    pixel_array_numpy = ds.pixel_array
    if PNG == False:
        image = image.replace('.dcm', '.jpg')
    else:
        image = image.replace('.dcm', '.png')
    cv2.imwrite(os.path.join(jpg_folder_path, image), pixel_array_numpy)
    if n % 50 == 0:
        print('{} image converted'.format(n))

In [ ]:
folder_path = "images/jpeg"
train_path = "images/jpeg/train"

In [ ]:
# make it True if you want in PNG format
PNG = True
# Specify the .dcm folder path
folder_path = "images/train"
# Specify the .jpg/.png folder path
jpg_folder_path = "images/png_train"
images_path = os.listdir(folder_path)
for n, image in enumerate(images_path):
    ds = dicom.dcmread(os.path.join(folder_path, image))
    pixel_array_numpy = ds.pixel_array
    if PNG == False:
        image = image.replace('.dcm', '.jpg')
    else:
        image = image.replace('.dcm', '.png')
    cv2.imwrite(os.path.join(jpg_folder_path, image), pixel_array_numpy)
    if n % 50 == 0:
        print('{} image converted'.format(n))

In [ ]:
# specify your image path
image_path = 'xray.dcm'
ds = dicom.dcmread(image_path)
plt.imshow( ds.pixel_array)

plt.show()

In [ ]:
import time
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

np.random.seed(123)

In [ ]:
df = pd.read_csv("data/train.csv")
df['target'] = df['target'].astype(str)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.20)
train_generator=datagen.flow_from_dataframe(
    dataframe=df,
    directory="images/jpeg",
    x_col="file",
    y_col="target",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(256,256))

In [ ]:
# automation.py
import os
from shutil import move

# directory path
test_dir = '/content/drive/MyDrive/Colab Notebooks/Melanoma/test'
train_dir = '/content/drive/MyDrive/Colab Notebooks/Melanoma/train'
test_zero = '/content/drive/MyDrive/Colab Notebooks/Melanoma/test/zero'
test_one = '/content/drive/MyDrive/Colab Notebooks/Melanoma/test/one'

# category wise file types
doc_types = ('.doc', '.docx', '.txt', '.pdf', '.xls', '.ppt', '.xlsx', '.pptx')
img_types = ('.jpg', '.jpeg', '.png', '.svg', '.gif', '.tif', '.tiff')
software_types = ('.exe', '.pkg', '.dmg')

def get_non_hidden_files_except_current_file(root_dir):
  return [f for f in os.listdir(root_dir) if os.path.isfile(f) and not f.startswith('.') and not f.__eq__(__file__)]

def move_files(files):
  for file in files:
    # file moved and overwritten if already exists
    if file in melanoma_ids:
      move(file, '{}/{}'.format(one, file))
      print('file {} moved to {}'.format(file, one))
    elif file in non_melanoma_ids:
      move(file, '{}/{}'.format(zero, file))
      print('file {} moved to {}'.format(file, zero))
    elif file.endswith(".jpg"):
 b 

if __name__ == "__main__":
files = get_non_hidden_files_except_current_file(root_dir)
move_files(files)